<a href="https://colab.research.google.com/github/peco-glhf/ai_writer/blob/main/audio_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# 必要なライブラリをインストール
!apt install -y ffmpeg
!pip install noisereduce librosa soundfile pydub

# 各種ライブラリをインポート
import os
import datetime
from google.colab import files
import noisereduce as nr
import librosa
import soundfile as sf
from pydub import AudioSegment, effects

# ベースフォルダを指定（プロジェクト単位のルート）
base_folder = '/content/drive/MyDrive/CLWORK/BL/音声処理結果/音声処理'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
# アップロード処理
uploaded = files.upload()
input_filename = list(uploaded.keys())[0]
file_basename = os.path.splitext(input_filename)[0]  # 拡張子なしファイル名


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# タイムスタンプ付きの一意フォルダを作成
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
drive_folder = f'{base_folder}/{file_basename}_{timestamp}'
os.makedirs(drive_folder, exist_ok=True)

print(f"処理結果は以下のフォルダに保存されます: {drive_folder}")

# ファイルを一時保存
drive_audio_path = f'{drive_folder}/{input_filename}'
with open(input_filename, 'rb') as f_in:
    with open(drive_audio_path, 'wb') as f_out:
        f_out.write(f_in.read())

# 音声をロード（モノラル, 16kHz）
audio_data, rate = librosa.load(drive_audio_path, sr=16000)

# ノイズ除去（冒頭2秒）
noise_sample = audio_data[0:rate*2]
reduced_noise_audio = nr.reduce_noise(
    y=audio_data,
    y_noise=noise_sample,
    sr=rate,
    stationary=True
)

# ノイズ除去ファイル保存
drive_clean_path = f'{drive_folder}/clean_{file_basename}.wav'
sf.write(drive_clean_path, reduced_noise_audio, rate)

# 音量正規化
audio_segment = AudioSegment.from_wav(drive_clean_path)
normalized_audio = effects.normalize(audio_segment)

# 最終ファイル保存
drive_final_path = f'{drive_folder}/final_{file_basename}.wav'
normalized_audio.export(drive_final_path, format='wav')

# 出力確認
print("✅ 音声処理完了")
print(f"📁 オリジナル: {drive_audio_path}")
print(f"📁 ノイズ除去済: {drive_clean_path}")
print(f"📁 最終出力: {drive_final_path}")